In [87]:
# General imports
import sys
sys.path.append("..")

import time
import numpy as np
import pandas as pd
import datetime
from scipy import sparse
from dateutil.relativedelta import relativedelta

TIME_REGEX = "%Y-%m-%d %H:%M:%S.%f"

## Load data

In [88]:
data_path = "../../data/x_nokc/CS-Question-Results6.csv"
df = pd.read_csv(data_path)
df.head()

,UserId,AttemptId,ObjectId,QuestionNumber,TemplateId,TemplateVersionId,Score,OutOf,EventTime,TimeCompleted,CollectionId,OrgUnitId,IsCorrect,OrgUnitCode,OrgUnitName,QText,AttemptStarted,AttemptCompleted,CollectionName
0,91920,4407040,4071650,1,41582,41753,4.0,4.0,2020-10-08 01:04:43.013,2020-10-09 02:49:38.613,358348,522926,1,CS-320-J1906-OL-TRAD-UG.20EW1,CS-320-J1906 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-10-08 01:00:40.677,2020-10-09 02:49:41.767,2-1 Quiz: Testing Techniques
1,146213,4353694,4066771,1,41582,41753,4.0,4.0,2020-09-28 04:12:43.203,2020-09-28 04:26:05.870,358039,522939,1,CS-320-T1592-OL-TRAD-UG.20EW1,CS-320-T1592 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-28 04:11:15.687,2020-09-28 04:26:07.610,2-1 Quiz: Testing Techniques
2,237884,4348067,4071650,1,41582,41753,4.0,4.0,2020-09-27 19:19:58.530,2020-09-27 19:25:55.180,358348,522926,1,CS-320-J1906-OL-TRAD-UG.20EW1,CS-320-J1906 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-27 19:19:38.347,2020-09-27 19:25:56.670,2-1 Quiz: Testing Techniques
3,183317,4340813,4141045,1,41582,41753,4.0,4.0,2020-09-26 01:27:10.560,2020-09-26 01:32:00.460,361937,522933,1,CS-320-T1591-OL-TRAD-UG.20EW1,CS-320-T1591 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-26 01:27:03.100,2020-09-26 01:32:03.010,2-1 Quiz: Testing Techniques
4,111146,4318799,4066771,1,41582,41753,4.0,4.0,2020-09-21 04:23:35.520,2020-09-21 04:29:43.670,358039,522939,1,CS-320-T1592-OL-TRAD-UG.20EW1,CS-320-T1592 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-21 04:17:58.057,2020-09-21 04:29:58.070,2-1 Quiz: Testing Techniques


In [89]:
pd.set_option('display.max_rows', 300)


In [90]:
df.dtypes

UserId                 int64
AttemptId              int64
ObjectId               int64
QuestionNumber         int64
TemplateId             int64
TemplateVersionId      int64
Score                float64
OutOf                float64
EventTime             object
TimeCompleted         object
CollectionId           int64
OrgUnitId              int64
IsCorrect              int64
OrgUnitCode           object
OrgUnitName           object
QText                 object
AttemptStarted        object
AttemptCompleted      object
CollectionName        object
dtype: object

In [91]:
df.isna().any()

UserId               False
AttemptId            False
ObjectId             False
QuestionNumber       False
TemplateId           False
TemplateVersionId    False
Score                 True
OutOf                False
EventTime             True
TimeCompleted         True
CollectionId         False
OrgUnitId            False
IsCorrect            False
OrgUnitCode          False
OrgUnitName          False
QText                False
AttemptStarted       False
AttemptCompleted      True
CollectionName       False
dtype: bool

In [92]:
df['ImputedEventTime'] = df['EventTime']

In [93]:
df['ImputedEventTime'].fillna(df['TimeCompleted'], inplace=True)

In [94]:
df['ImputedEventTime'].fillna(df['AttemptCompleted'], inplace=True)

In [95]:
df['ImputedEventTime'].fillna(df['AttemptStarted'], inplace=True)

In [96]:
def format_unix_time(row):
    return datetime.datetime.strptime(row['ImputedEventTime'], TIME_REGEX).timestamp()

In [97]:
df['unix_time'] = df.apply(format_unix_time, axis=1)

In [98]:
df.head()


,UserId,AttemptId,ObjectId,QuestionNumber,TemplateId,TemplateVersionId,Score,OutOf,EventTime,TimeCompleted,...,OrgUnitId,IsCorrect,OrgUnitCode,OrgUnitName,QText,AttemptStarted,AttemptCompleted,CollectionName,ImputedEventTime,unix_time
0,91920,4407040,4071650,1,41582,41753,4.0,4.0,2020-10-08 01:04:43.013,2020-10-09 02:49:38.613,...,522926,1,CS-320-J1906-OL-TRAD-UG.20EW1,CS-320-J1906 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-10-08 01:00:40.677,2020-10-09 02:49:41.767,2-1 Quiz: Testing Techniques,2020-10-08 01:04:43.013,1.602144e+09
1,146213,4353694,4066771,1,41582,41753,4.0,4.0,2020-09-28 04:12:43.203,2020-09-28 04:26:05.870,...,522939,1,CS-320-T1592-OL-TRAD-UG.20EW1,CS-320-T1592 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-28 04:11:15.687,2020-09-28 04:26:07.610,2-1 Quiz: Testing Techniques,2020-09-28 04:12:43.203,1.601292e+09
2,237884,4348067,4071650,1,41582,41753,4.0,4.0,2020-09-27 19:19:58.530,2020-09-27 19:25:55.180,...,522926,1,CS-320-J1906-OL-TRAD-UG.20EW1,CS-320-J1906 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-27 19:19:38.347,2020-09-27 19:25:56.670,2-1 Quiz: Testing Techniques,2020-09-27 19:19:58.530,1.601260e+09
3,183317,4340813,4141045,1,41582,41753,4.0,4.0,2020-09-26 01:27:10.560,2020-09-26 01:32:00.460,...,522933,1,CS-320-T1591-OL-TRAD-UG.20EW1,CS-320-T1591 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-26 01:27:03.100,2020-09-26 01:32:03.010,2-1 Quiz: Testing Techniques,2020-09-26 01:27:10.560,1.601109e+09
4,111146,4318799,4066771,1,41582,41753,4.0,4.0,2020-09-21 04:23:35.520,2020-09-21 04:29:43.670,...,522939,1,CS-320-T1592-OL-TRAD-UG.20EW1,CS-320-T1592 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-21 04:17:58.057,2020-09-21 04:29:58.070,2-1 Quiz: Testing Techniques,2020-09-21 04:23:35.520,1.600687e+09


In [99]:
df['timestamp'] = df['unix_time'] - np.min(df['unix_time'])

In [100]:
df.head()

,UserId,AttemptId,ObjectId,QuestionNumber,TemplateId,TemplateVersionId,Score,OutOf,EventTime,TimeCompleted,...,IsCorrect,OrgUnitCode,OrgUnitName,QText,AttemptStarted,AttemptCompleted,CollectionName,ImputedEventTime,unix_time,timestamp
0,91920,4407040,4071650,1,41582,41753,4.0,4.0,2020-10-08 01:04:43.013,2020-10-09 02:49:38.613,...,1,CS-320-J1906-OL-TRAD-UG.20EW1,CS-320-J1906 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-10-08 01:00:40.677,2020-10-09 02:49:41.767,2-1 Quiz: Testing Techniques,2020-10-08 01:04:43.013,1.602144e+09,8.734495e+07
1,146213,4353694,4066771,1,41582,41753,4.0,4.0,2020-09-28 04:12:43.203,2020-09-28 04:26:05.870,...,1,CS-320-T1592-OL-TRAD-UG.20EW1,CS-320-T1592 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-28 04:11:15.687,2020-09-28 04:26:07.610,2-1 Quiz: Testing Techniques,2020-09-28 04:12:43.203,1.601292e+09,8.649223e+07
2,237884,4348067,4071650,1,41582,41753,4.0,4.0,2020-09-27 19:19:58.530,2020-09-27 19:25:55.180,...,1,CS-320-J1906-OL-TRAD-UG.20EW1,CS-320-J1906 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-27 19:19:38.347,2020-09-27 19:25:56.670,2-1 Quiz: Testing Techniques,2020-09-27 19:19:58.530,1.601260e+09,8.646027e+07
3,183317,4340813,4141045,1,41582,41753,4.0,4.0,2020-09-26 01:27:10.560,2020-09-26 01:32:00.460,...,1,CS-320-T1591-OL-TRAD-UG.20EW1,CS-320-T1591 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-26 01:27:03.100,2020-09-26 01:32:03.010,2-1 Quiz: Testing Techniques,2020-09-26 01:27:10.560,1.601109e+09,8.630950e+07
4,111146,4318799,4066771,1,41582,41753,4.0,4.0,2020-09-21 04:23:35.520,2020-09-21 04:29:43.670,...,1,CS-320-T1592-OL-TRAD-UG.20EW1,CS-320-T1592 Software Test Automation& QA 20EW1,<p>True or false: Under static software testin...,2020-09-21 04:17:58.057,2020-09-21 04:29:58.070,2-1 Quiz: Testing Techniques,2020-09-21 04:23:35.520,1.600687e+09,8.588808e+07


In [101]:
np.min(df['timestamp'])


0.0

In [102]:
df.describe()


,UserId,AttemptId,ObjectId,QuestionNumber,TemplateId,TemplateVersionId,Score,OutOf,CollectionId,OrgUnitId,IsCorrect,unix_time,timestamp
count,50739.000000,5.073900e+04,5.073900e+04,50739.000000,50739.000000,50739.000000,50576.000000,50739.000000,50739.000000,50739.000000,50739.000000,5.073900e+04,5.073900e+04
mean,167757.346400,3.922381e+06,3.762865e+06,5.732533,112093.452571,124446.378762,2.465303,2.885926,330296.010150,482341.034510,0.843749,1.593612e+09,7.881252e+07
std,85142.112263,1.107120e+06,1.080996e+06,3.157266,61538.136946,67061.550881,1.364496,0.941124,92351.374269,151755.202804,0.363096,1.856686e+07,1.856686e+07
min,9755.000000,5.621600e+04,7.079400e+04,1.000000,41582.000000,41753.000000,0.000000,1.000000,4616.000000,17673.000000,0.000000,1.514799e+09,0.000000e+00
25%,87525.000000,3.500973e+06,3.412278e+06,3.000000,58684.000000,60983.000000,2.000000,2.000000,299812.000000,428704.000000,1.000000,1.588896e+09,7.409687e+07
50%,162421.000000,4.206119e+06,4.066473e+06,6.000000,58723.000000,135506.000000,2.500000,3.000000,358018.000000,524231.000000,1.000000,1.599457e+09,8.465802e+07
75%,239937.000000,4.725823e+06,4.406688e+06,8.000000,164480.000000,181017.000000,3.750000,3.750000,385240.000000,576870.000000,1.000000,1.605846e+09,9.104628e+07
max,374372.000000,5.425883e+06,5.247347e+06,15.000000,207440.000000,226348.000000,4.000000,4.000000,454338.000000,701407.000000,1.000000,1.615216e+09,1.004167e+08


In [103]:
df.nunique()


UserId                1557
AttemptId             5001
ObjectId              1594
QuestionNumber          15
TemplateId             121
TemplateVersionId      127
Score                   10
OutOf                    8
EventTime            50135
TimeCompleted        36711
CollectionId           158
OrgUnitId              108
IsCorrect                2
OrgUnitCode            108
OrgUnitName            108
QText                  122
AttemptStarted        5001
AttemptCompleted      4984
CollectionName          11
ImputedEventTime     50591
unix_time            50591
timestamp            50591
dtype: int64

In [60]:
df['IsCorrect'].sum()


14254

In [ ]:
def partial_score(row):
    return row['score'] > 0

In [61]:
df.head(300)


,UserId,AttemptId,ObjectId,QuestionNumber,TemplateId,TemplateVersionId,Score,OutOf,EventTime,TimeCompleted,...,OrgUnitId,IsCorrect,OrgUnitCode,OrgUnitName,QText,AttemptStarted,AttemptCompleted,ImputedEventTime,unix_time,timestamp
0,109401,3581924,3416460,1,41582,41753,4.0,4.0,2020-05-18 00:41:01.253,2020-05-18 00:54:33.210,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-18 00:40:49.893,2020-05-18 00:54:37.853,2020-05-18 00:41:01.253,1.589788e+09,7.498833e+07
1,140167,3582804,3416460,1,41582,41753,4.0,4.0,2020-05-18 01:42:55.163,2020-05-18 01:53:33.840,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-18 01:42:44.397,2020-05-18 01:53:35.777,2020-05-18 01:42:55.163,1.589791e+09,7.499204e+07
2,186568,3583100,3416460,1,41582,41753,4.0,4.0,2020-05-18 02:07:24.440,2020-05-18 02:27:59.613,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-18 02:04:39.403,2020-05-18 02:28:03.193,2020-05-18 02:07:24.440,1.589793e+09,7.499351e+07
3,100851,3579747,3416460,1,41582,41753,4.0,4.0,2020-05-17 21:31:15.233,2020-05-17 21:34:29.877,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-17 21:31:07.517,2020-05-17 21:34:32.797,2020-05-17 21:31:15.233,1.589776e+09,7.497694e+07
4,220756,3579563,3416460,1,41582,41753,4.0,4.0,2020-05-17 21:14:49.527,2020-05-17 21:19:18.683,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-17 21:12:39.743,2020-05-17 21:19:21.577,2020-05-17 21:14:49.527,1.589775e+09,7.497596e+07
5,195417,3577282,3416460,1,41582,41753,4.0,4.0,2020-05-17 17:16:46.617,2020-05-17 17:25:48.153,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-17 17:16:36.227,2020-05-17 17:25:50.823,2020-05-17 17:16:46.617,1.589761e+09,7.496167e+07
6,182857,3576988,3416460,1,41582,41753,4.0,4.0,2020-05-17 16:41:13.560,2020-05-17 18:54:29.083,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-17 16:40:20.167,2020-05-17 18:54:35.600,2020-05-17 16:41:13.560,1.589759e+09,7.495954e+07
7,185548,3575177,3416460,1,41582,41753,4.0,4.0,2020-05-17 06:54:41.180,2020-05-17 07:06:26.840,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-17 06:54:34.897,2020-05-17 07:06:29.090,2020-05-17 06:54:41.180,1.589724e+09,7.492435e+07
8,125858,3575587,3416460,1,41582,41753,4.0,4.0,2020-05-17 12:52:58.890,2020-05-17 13:23:37.290,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-17 12:51:42.590,2020-05-17 13:23:39.773,2020-05-17 12:52:58.890,1.589745e+09,7.494585e+07
9,89669,3576291,3416460,1,41582,41753,4.0,4.0,2020-05-17 15:10:28.270,2020-05-17 15:11:23.240,...,428700,0,CS-320-J5705-OL-TRAD-UG.20EW5,CS-320-J5705 Software Test Automation& QA 20EW5,<p>True or false: Under static software testin...,2020-05-17 15:09:54.347,2020-05-17 15:11:32.677,2020-05-17 15:10:28.270,1.589753e+09,7.495410e+07


In [104]:
df['QText'].unique()


array(['<p>True or false: Under static software testing, code is not executed.</p>',
       '<p>Dynamic software testing code is executed and checks software behavioral function.</p>',
       '<p>Black-box testing is a dynamic software testing technique.</p>',
       '<p>Under white-box testing, the inside details of the software are not visible.</p>',
       '<p>In unit testing, units or modules are tested by the developers.</p>',
       '<p>Integrated testing is grouping modules together for the purpose of determining if the modules function correctly together.</p>',
       '<p><span data-header-only="false" tab_caption="Notes" tab_caption_raw="Notes" id="section_tab.1a8dc271db576200c33c7bec0f9619bd" data-section-id="1a8dc271db576200c33c7bec0f9619bd"><span id="section-1a8dc271db576200c33c7bec0f9619bd" data-header-only="false">Integration testing is grouping modules together for the purpose of determining if the modules function correctly together.</span></span></p>',
       '<p>Under

'CS-320'